In [1]:
import gpflow
import numpy as np
import math
import tensorflow as tf
sess = tf.Session()

In [2]:
import get_history as gh

In [20]:



class CustomKernel(gpflow.kernels.Kernel):
    def __init__(self,input_dim,variance=1.0, lengthscales=1.0,
                 active_dims=None, ARD=None, name=None):
        super().__init__(input_dim)

        self.sigma_u = tf.constant([[100.0,10.0],[10.0,100.0]])
        
        self.sigma_theta = tf.constant([[100.0,10.0,10.0],[10.0,100.0,10.0],\
                                    [10.0,10.0,100.0]])
        
        #gpflow.Param(1.0, transform=gpflow.transforms.positive,
                                  #dtype=gpflow.settings.float_type)
        
        self.sigma_v =  tf.constant([[100.0,0.1],[0.1,100.0]])
        #gpflow.Param(1.0, transform=gpflow.transforms.positive,
                                  #dtype=gpflow.settings.float_type)
        
        ##will it freak out if there is no parameter?
        
    
    @staticmethod
    def rbf(self, X, X2):
        return tf.exp(-(X-X2) / 2.2)
    
    @gpflow.params_as_tensors
    def _scaled_square_dist(self, X, X2):
        """
        Assumes each X has the form: [phi(state_action),psi(state_action),user_index,user_day_in_study_index]
        
        
        """
        X = X / self.lengthscales
        Xs = tf.reduce_sum(tf.square(X), axis=-1, keepdims=True)

        if X2 is None:
            dist = -2 * tf.matmul(X, X, transpose_b=True)
            dist += Xs + tf.matrix_transpose(Xs)
            return dist

        X2 = X2 / self.lengthscales
        X2s = tf.reduce_sum(tf.square(X2), axis=-1, keepdims=True)
        dist = -2 * tf.matmul(X, X2, transpose_b=True)
        dist += Xs + tf.matrix_transpose(X2s)
        return dist
    
    @staticmethod
    def _clipped_sqrt(r2):
        # Clipping around the (single) float precision which is ~1e-45.
        return tf.sqrt(tf.maximum(r2, 1e-40))
    
    def scaled_square_dist(self, X, X2):  # pragma: no cover
        return self._scaled_square_dist(X, X2)
    
    @gpflow.params_as_tensors
    def K_r2(self, X, X2):
        return self.variance * tf.exp(-r2 / 2.)
    
    @gpflow.params_as_tensors
    def K(self, X, X2=None, presliced=False):
        
        #feature_vector_one = X[0:4]
        #time_index_one = X[4:5]
        #user_index_one = X[5:6]
        
        
        #if X2!=None:
            #feature_vector_two = X2[0:4]
            #time_index_two = X2[4:5]
            #user_index_two = X2[5:6]

        user_id_one = X[-2]
        day_one = X[-1]
        f_one = X[0:3]
        g_one = X[3:5]
        rho_term = 1
        
        result = 1
        
        if X2!=None:
            user_id_two = X2[-2]
            day_two = X2[-1]
            f_two = X2[0:3]
            g_two = X2[3:5]
        
        
            rho_term = self.rbf(X,X2) 
            
            baselines = tf.matmul(f_one,self.sigma_theta,a_transpose=True)
            baselines = tf.matmul(baselines,f_two)
            
            print(baselines)
            
            if user_one==user_two:
                user_term = self.sigma_u
            else:
                user_term = tf.constant([0.0,0.0,0.0,0.0])
            
            effect_term = tf.scalar_mul(rho_term,self.sigma_v)
            effect_term = tf.add(user_term,effect_term)
            effects = tf.matmul(g_one,effect_term,a_transpose=True)
            effects = tf.matmul(effects,g_two)
            resulf = tf.add(baselines,effects)
            #stuff to return
            
            
      
        
        return result
       
        
        #self.variance * tf.minimum(X, tf.transpose(X2))

    @gpflow.params_as_tensors
    def Kdiag(self, X, presliced=False):
        return tf.fill(tf.shape(X)[:-1], tf.squeeze(self.variance))

{0: 1,
 2: 2,
 11: 3,
 15: 4,
 19: 5,
 26: 6,
 35: 7,
 37: 8,
 43: 9,
 44: 10,
 47: 11,
 58: 12,
 59: 13,
 74: 14,
 78: 15,
 79: 16,
 80: 17,
 82: 18,
 91: 19,
 93: 20,
 1: 1,
 3: 2,
 6: 3,
 7: 4,
 8: 5,
 12: 6,
 14: 7,
 22: 8,
 23: 9,
 27: 10,
 28: 11,
 32: 12,
 41: 13,
 46: 14,
 48: 15,
 63: 16,
 64: 17,
 66: 18,
 68: 19,
 76: 20,
 83: 21,
 92: 22,
 94: 23,
 4: 1,
 9: 2,
 16: 3,
 21: 4,
 24: 5,
 25: 6,
 33: 7,
 34: 8,
 38: 9,
 40: 10,
 51: 11,
 52: 12,
 70: 13,
 71: 14,
 72: 15,
 81: 16,
 87: 17,
 88: 18,
 89: 19,
 90: 20,
 99: 21,
 5: 1,
 10: 2,
 17: 3,
 18: 4,
 20: 5,
 30: 6,
 39: 7,
 45: 8,
 49: 9,
 50: 10,
 54: 11,
 56: 12,
 57: 13,
 61: 14,
 62: 15,
 65: 16,
 67: 17,
 69: 18,
 73: 19,
 75: 20,
 84: 21,
 86: 22,
 95: 23,
 96: 24,
 98: 25,
 13: 1,
 29: 2,
 31: 3,
 36: 4,
 42: 5,
 53: 6,
 55: 7,
 60: 8,
 77: 9,
 85: 10,
 97: 11}

In [8]:
##need to reformat history
def put_x_together(h):
    X = []
    
    for i in range(len(h.states)):
        state_vector = h.states[i]
        state_vector.append(h.users[i])
        state_vector.append(h.day_function[i])
        X.append(state_vector)
    return X


In [11]:
history = gh.get_history(5,100,5)

In [12]:
X = put_x_together(history)

In [14]:
X = np.array(X)

In [15]:
y = np.array([[r] for r in history.rewards])

In [21]:
k =  CustomKernel(5)
m = gpflow.models.GPR(X,y, kern=k)

In [22]:
gpflow.train.ScipyOptimizer().minimize(m,session=sess)

INFO:tensorflow:Optimization terminated with:
  Message: b'CONVERGENCE: NORM_OF_PROJECTED_GRADIENT_<=_PGTOL'
  Objective function value: 643.655810
  Number of iterations: 36
  Number of functions evaluations: 37


INFO:tensorflow:Optimization terminated with:
  Message: b'CONVERGENCE: NORM_OF_PROJECTED_GRADIENT_<=_PGTOL'
  Objective function value: 643.655810
  Number of iterations: 36
  Number of functions evaluations: 37


In [23]:
m.as_pandas_table()

,class,prior,transform,trainable,shape,fixed_shape,value
GPR/likelihood/variance,Parameter,None,+ve,True,(),True,22719.843323996523


In [22]:
def obtain_kernel_and_sigma_u(all_xs,all_ys):
    k =  CustomKernel(all_xs.shape[1])
    m = gpflow.models.GPR(all_xs, all_ys, kern=k)
    gpflow.train.ScipyOptimizer().minimize(m,session=sess)
    sigma_u = m.kern.sigma_u.value
    cov = np.zeros((all_xs.shape[0],all_xs.shape[0]))
    print(m.as_pandas_table())
    for i in range(all_xs.shape[0]):
        for j in range(all_xs.shape[0]):
            term = m.kern.K(all_xs[i],all_xs[j])
            cov[i][j]=term.eval(session=sess)
    return cov,sigma_u

In [23]:
obtain_kernel_and_sigma_u(X,y)


INFO:tensorflow:Optimization terminated with:
  Message: b'CONVERGENCE: NORM_OF_PROJECTED_GRADIENT_<=_PGTOL'
  Objective function value: 0.312148
  Number of iterations: 5
  Number of functions evaluations: 9


INFO:tensorflow:Optimization terminated with:
  Message: b'CONVERGENCE: NORM_OF_PROJECTED_GRADIENT_<=_PGTOL'
  Objective function value: 0.312148
  Number of iterations: 5
  Number of functions evaluations: 9


                             class prior transform  trainable shape  \
GPR/kern/lengthscales    Parameter  None       +ve       True    ()   
GPR/kern/sigma_u         Parameter  None       +ve       True    ()   
GPR/likelihood/variance  Parameter  None       +ve       True    ()   

                         fixed_shape                 value  
GPR/kern/lengthscales           True                   1.0  
GPR/kern/sigma_u                True    0.1500026161960994  
GPR/likelihood/variance         True  0.020000046520069675  


(array([[0.15000262, 0.15000262],
        [0.15000262, 0.15000262]]), array(0.15000262))

In [118]:
#one = tf.constant([0,0])
#two = tf.constant([1,1])
inp= tf.constant([[1.0, 2, 3], [4, 5, 6.0], [7.0, 8, 9]])
tf.norm([inp,inp],ord=2)


<tf.Tensor 'norm_9/Squeeze:0' shape=() dtype=float32>

In [16]:

gr = m.kern.K(X,X)

In [19]:
X.shape

(2, 6)

In [20]:
gr.eval()

1.0